[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/028_pytorch_nn/pytorch_nn.ipynb)

In [1]:
import torch #Importar PyTorch

import pandas as pd #Manipulación y análisis de datos tabulares

from sklearn.preprocessing import LabelEncoder #Codificar etiquetas categóricas en valores numéricos.


Definir las dimensiones de entrada (D_in), la capa oculta (H) y la dimensión de salida (D_out) para una red neuronal.

In [2]:
D_in, H, D_out = 784, 100, 10

# Crear un modelo de red neuronal secuencial utilizando PyTorch.
model = torch.nn.Sequential(
    # Capa lineal que toma D_in entradas y produce H salidas, seguida de una función de activación ReLU.
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    # Otra capa lineal que toma las H salidas anteriores y produce D_out salidas (en este caso, 10 salidas para la clasificación de 10 clases).
    torch.nn.Linear(H, D_out),
)


Realizar una inferencia en el modelo de red neuronal 'model' utilizando datos de entrada aleatorios con forma (64, 784).

Mostrar la forma de las salidas calculadas por el modelo. Esto te dará la forma de los resultados de la inferencia.
En este caso, 'outputs' contendrá un tensor con forma (64, 10), ya que el modelo tiene una capa de salida con 10 unidades.

In [3]:
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

Imprimir los valores de la primera fila (primer ejemplo) de las salidas 'outputs'.
'outputs' es un tensor con forma (64, 10), por lo que esto imprimirá los 10 valores correspondientes al primer ejemplo.

In [4]:
print(outputs[0][:])

tensor([-0.0783,  0.2189, -0.1786, -0.3281,  0.1013,  0.1278,  0.2317, -0.1106,
        -0.3287,  0.1816], grad_fn=<SliceBackward0>)


Mover el modelo 'model' desde la CPU a la GPU (Dispositivo de Unidad de Procesamiento Gráfico) utilizando el método 'to'.

Después de esta operación, las operaciones de la red neuronal se realizarán en la GPU, lo que puede acelerar el entrenamiento y la inferencia si se dispone de una GPU compatible.


In [5]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

Vamos a ver ahora como entrenar este modelo con el dataset MNIST.

In [7]:
#Tratamiento y manejo de datos
data = pd.read_csv('Glyphs_updated.csv')

# Eliminar todas las columnas con valores NaN
data = data.dropna(axis=1)

# Llenar los valores faltantes con un valor específico, como cero (0)
data = data.fillna(0)

Y = data['label']
X = data.drop(columns=['label'])

X = X.apply(pd.to_numeric, errors='coerce')
X = X.fillna(0)

letras = data['label'].unique()  # Obtiene todas las letras únicas en la columna 'label'
mapeo_letras = {letra: i for i, letra in enumerate(letras)}

X.shape, Y.shape

((54894, 784), (54894,))

In [8]:
import numpy as np

# Crear un arreglo NumPy 'x_2' a partir de la variable 'X'.
x_2 = np.array(X)

# Mapear las etiquetas de clase en 'data' utilizando el diccionario 'mapeo_letras' y almacenar el resultado en 'y_2'.
data['label'] = data['label'].map(mapeo_letras)
y_2 = np.array(data['label'].values)

# Normalizar los datos dividiendo cada valor en 'x_2' entre 255. Esto escala los valores de píxeles al rango [0, 1].
X_train = x_2[:34894] / 255.  # Conjunto de entrenamiento
X_test = x_2[20000:] / 255.   # Conjunto de prueba

# Dividir las etiquetas en conjuntos de entrenamiento y prueba.
y_train = y_2[:34894]  # Etiquetas de entrenamiento
y_test = y_2[:20000]   # Etiquetas de prueba

Definir una función de softmax que toma un tensor 'x' como entrada y calcula las probabilidades de clase softmax.
Definir una función de pérdida de entropía cruzada que toma las salidas 'output' del modelo y las etiquetas 'target' como entrada.

In [9]:
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1, keepdims=True)

def cross_entropy(output, target):
    # Calcular los log-softmax de las salidas para obtener los logits de las clases correctas.
    logits = output[torch.arange(len(output)), target]

    # Calcular la pérdida de entropía cruzada mediante la fórmula -log(p(y_true)) + log(sum(exp(p(y_i)))).
    loss = -logits + torch.log(torch.sum(torch.exp(output), axis=-1))

    # Calcular el promedio de la pérdida a través de los ejemplos en el lote.
    loss = loss.mean()

    return loss

Comprobar si la GPU (Unidad de Procesamiento Gráfico) está disponible para su uso con PyTorch.

In [10]:
torch.cuda.is_available()

True

In [11]:
print(X)

       1  2  3  4  5  6  7  8  9  10  ...  775  776  777  778  779  780  781  \
0      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
1      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
2      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
3      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
4      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
...   .. .. .. .. .. .. .. .. ..  ..  ...  ...  ...  ...  ...  ...  ...  ...   
54889  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
54890  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
54891  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
54892  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
54893  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   

       782  783  784  
0        0    0 

In [12]:
print(y_train)
y_train.dtype

[  0   1   2 ...  98 110 111]


dtype('int64')

Convertir los datos de entrenamiento 'X_train' y 'y_train' a tensores de PyTorch y copiarlos a la GPU.

In [13]:
X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# Configurar el número de épocas (epochs) y la tasa de aprendizaje (lr).
epochs = 100
lr = 0.8

# Especificar la frecuencia con la que se registrará el valor de la función de pérdida durante el entrenamiento.
log_each = 10

# Lista 'l' para almacenar los valores de pérdida en cada época.
l = []

# Bucle de entrenamiento.
for e in range(1, epochs + 1):
    # Realizar el pase hacia adelante (forward pass) a través del modelo para obtener las predicciones 'y_pred'.
    y_pred = model(X_t)

    # Calcular la función de pérdida utilizando las predicciones y las etiquetas de entrenamiento.
    loss = cross_entropy(y_pred, Y_t)

    # Registrar el valor de pérdida en la lista 'l'.
    l.append(loss.item())

    # Establecer los gradientes en cero para evitar acumulaciones.
    model.zero_grad()

    # Realizar el pase hacia atrás (backward pass) para calcular los gradientes automáticamente.
    loss.backward()

    # Actualizar los pesos del modelo utilizando el descenso de gradiente.
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

    # Limpiar la caché de la GPU para liberar memoria.
    torch.cuda.empty_cache()

    # Imprimir el valor de pérdida cada 'log_each' épocas.
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")


RuntimeError: ignored

Importar la función accuracy_score de Scikit-Learn para evaluar la precisión de las predicciones.


In [14]:

from sklearn.metrics import accuracy_score

# Definir una función 'evaluate' que toma un conjunto de datos 'x' como entrada y realiza evaluación utilizando el modelo.
def evaluate(x):
    # Cambiar el modo del modelo a evaluación (no entrenamiento).
    model.eval()

    # Obtener las predicciones 'y_pred' del modelo para el conjunto de datos 'x'.
    y_pred = model(x)

    # Aplicar la función de softmax a las predicciones 'y_pred' para obtener probabilidades de clase.
    y_probas = softmax(y_pred)

    # Encontrar la etiqueta de clase predicha tomando el índice del valor máximo en las probabilidades.
    return torch.argmax(y_probas, axis=1)

# Llamar a la función 'evaluate' con el conjunto de prueba 'X_test', convertir las predicciones a tensores en la GPU y calcular la precisión.
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())

# Calcular la precisión comparando las etiquetas reales 'y_test' con las etiquetas predichas 'y_pred'.
accuracy = accuracy_score(y_test, y_pred.cpu().numpy())

# La variable 'accuracy' contiene la precisión del modelo en el conjunto de prueba.


RuntimeError: ignored